In [23]:
#import streamlit as st
from os import X_OK
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

import joblib

from scipy.sparse import csr_matrix

In [24]:
import googlemaps

In [ ]:
def latitude(column):
    key = pd.read_csv("../raw_data/api_key_vb.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lat'])

def longitude(column):
    key = pd.read_csv("../raw_data/api_key_vb.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lng'])

def get_fitted_pipe(X, y):
    coordinate_transformer = SimpleImputer(strategy="most_frequent")
    price_transformer = SimpleImputer(strategy="mean")
    cat_transformer = OneHotEncoder(handle_unknown='ignore')
    preproc = make_column_transformer((price_transformer, ['price']),
                                       (coordinate_transformer, ['latitude', 'longitude']),
                                        (cat_transformer, ['neighborhood', 'type']), remainder='passthrough')
    pipe = make_pipeline(preproc)
    pipe.fit_transform(X) #y_class)
    return pipe

def target_X(type_of_food, price, neighborhood, takeaway, latitude, longitude):
    new_df=pd.DataFrame(columns=['type','price','latitude','longitude', 'dine_in', 'takeaway','delivery', 'neighborhood'])
    new_row = {'type':type_of_food, 'price': price, 'latitude':latitude,'longitude':longitude,'takeaway':takeaway,'dine_in':1,'delivery':1,  'neighborhood': neighborhood}
    X_user= new_df.append(new_row, ignore_index=True)
    return X_user

def build_y(y):
    y_class=pd.cut(x=y, bins=[0,4, 5],
                        labels=[ "below_average", "above_average"])
    return y_class
#target_X(type_of_food = 'brunch', price = '2', neighborhood = 'Graça', takeaway = 1, latitude =38.714376, longitude = -9.130176)
# target X function not yet finalised, name, and other factors to be seen

def train_model(pipeline, X, y_class):

    knn = KNeighborsClassifier(n_neighbors = 5,weights= 'uniform')
    knn.fit(pipeline.fit_transform(X), y_class)
    filename = 'knn_model.joblib'
    joblib.dump(knn, filename)



def predict(X_user_transformed):

    loaded_model = joblib.load('knn_model.joblib')
    print('model loaded')
    y_probxgb = loaded_model.predict_proba(X_user_transformed)
    print(y_probxgb[0][1])
    if y_probxgb[0][1]< 0.8:
            pred =  'below_average'
    else:
            pred = 'above average'
    neighbors =  loaded_model.kneighbors(X_user_transformed, n_neighbors=10, return_distance=False)
    print(neighbors)
    return neighbors, pred




In [2]:
def restaurant_data():

    data = pd.read_csv('../small_business/data/restaurants.csv')
    data = data.drop(columns = 'Unnamed: 0')
    X = data.drop(columns=['rating','name', 'address', 'label', 'curb_pickup', 'drive_through', 'postal_code', 'no_del_exp', 'municipality', 'review_count'])
    y = data['rating']
    return X, y

In [12]:



def latitude(column):
    key = pd.read_csv("../raw_data/api_key_vb.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lat'])

def longitude(column):
    key = pd.read_csv("../raw_data/api_key_vb.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lng'])

def get_fitted_pipe(X, y):
    coordinate_transformer = SimpleImputer(strategy="most_frequent")
    price_transformer = SimpleImputer(strategy="mean")
    cat_transformer = OneHotEncoder(handle_unknown='ignore')
    preproc = make_column_transformer((price_transformer, ['price']),
                                       (coordinate_transformer, ['latitude', 'longitude']),
                                        (cat_transformer, ['neighborhood', 'type']), remainder='passthrough')
    pipe = make_pipeline(preproc)
    pipe.fit_transform(X) #y_class)
    return pipe

def target_X(type_of_food, price, neighborhood, takeaway, latitude, longitude):
    new_df=pd.DataFrame(columns=['type','price','latitude','longitude', 'dine_in', 'takeaway','delivery', 'neighborhood'])
    new_row = {'type':type_of_food, 'price': price, 'latitude':latitude,'longitude':longitude,'takeaway':takeaway,'dine_in':1,'delivery':1,  'neighborhood': neighborhood}
    X_user= new_df.append(new_row, ignore_index=True)
    return X_user

def build_y(y):
    y_class=pd.cut(x=y, bins=[0,4, 5],
                        labels=[ "below_average", "above_average"])
    return y_class
#target_X(type_of_food = 'brunch', price = '2', neighborhood = 'Graça', takeaway = 1, latitude =38.714376, longitude = -9.130176)
# target X function not yet finalised, name, and other factors to be seen

def train_model(pipeline, X, y_class):

    knn = KNeighborsClassifier(n_neighbors = 5,weights= 'uniform')
    knn.fit(pipeline.fit_transform(X), y_class)
    filename = 'knn_model.joblib'
    joblib.dump(knn, filename)



def predict(X_user_transformed):

    loaded_model = joblib.load('knn_model.joblib')
    print('model loaded')
    y_probxgb = loaded_model.predict_proba(X_user_transformed)
    print(y_probxgb[0][1])
    if y_probxgb[0][1]< 0.8:
            pred =  'below_average'
    else:
            pred = 'above average'
    neighbors =  loaded_model.kneighbors(X_user_transformed, n_neighbors=10, return_distance=False)
    print(neighbors)
    return neighbors, pred






In [13]:
 X, y = restaurant_data()

In [14]:
pipeline = get_fitted_pipe(X, y)

In [15]:
X_user = target_X(type_of_food = 'brunch', price = '2', neighborhood = 'Graça', takeaway = 1, latitude =38.714376, longitude = -9.130176)

In [16]:
X_user_transformed = pipeline.transform(X_user)

In [17]:
print(X_user_transformed.shape)

(1, 69)


In [18]:
y_class = build_y(y)

In [19]:
train_model(pipeline, X, y_class)

In [20]:
predict(X_user_transformed)

model loaded
0.0
[[ 32   0 161 633 284 495 556 332 428 574]]


(array([[ 32,   0, 161, 633, 284, 495, 556, 332, 428, 574]]), 'below_average')

In [21]:
csr_matrix.todense(X_user_transformed)

matrix([[ 2.      , 38.714376, -9.130176,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  1.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  1.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
          0.      ,  1.      ,  1.      ,  1.      ]])

In [ ]:
pipeline

In [ ]:
X_user

In [ ]:
!ls